<a href="https://colab.research.google.com/github/Gaurav37/Natural-Language-Processing/blob/main/Copy_of_PS1_Shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks such as summarization is hard enoough.

In [ ]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
import numpy as np

In [ ]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

This is where you will implement two functions to featurize the data:

In [ ]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
from sklearn.feature_extraction.text import CountVectorizer
def createBasicFeatures(corpus):
  #Your code here
  genres=[]
  for i in range(0,len(corpus)):
    if corpus[i]['genre'] == "tragedy":
      genres.append(0)
    else:
      genres.append(1)  
  #class conversion to int
  corpus_text=[]
  for i in range(0,len(corpus)):
    corpus_text.append(corpus[i]['text'])
  vec = CountVectorizer()
  texts=vec.fit_transform(corpus_text)#convert to sparse #Dense versus sparse
  vocab=vec.get_feature_names()
  #print("parameters",vec.get_params())

  return texts,genres,vocab

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createInterestingFeatures(corpus):
  #Your code here
  #Your code here
  from nltk import bigrams
  genres=[]
  for i in range(0,len(corpus)):
    if corpus[i]['genre'] == "tragedy":
      genres.append(0)
    else:
      genres.append(1)  
  #class conversion to int
  corpus_text=[]
  for i in range(0,len(corpus)):
    corpus_text.append(corpus[i]['text'])
  vec = CountVectorizer(ngram_range=(2,2))
  texts=vec.fit_transform(corpus_text)#convert to sparse #Dense versus sparse
  vocab=vec.get_feature_names()
  #print("parameters",vec.get_params())

  #Add to basic ..
  #Focussed bigrams, 
  #Remove unwanted features
  
  return texts,genres,vocab

In [ ]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  

In [ ]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [ ]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.807692
The most informative terms for pos are: ['our', 'him', 'rom', 'iago', 'thy', 'ham', 'imogen', 'his', 'what', 'brutus', 'lear', 'timon', 'preservers', 'preserver', 'preserved', 'preserv', 'preservative', 'preservation', 'presents', 'presentment']
The most informative terms for neg are: ['helena', 'prospero', 'sir', 'your', 'you', 'for', 'me', 'duke', 'of', 'love', 'preserver', 'preserved', 'preserve', 'preserv', 'preservation', 'preservers', 'presents', 'presentment', 'presently', 'presenting']
----------L2 Norm-----------
The model's average accuracy is 0.730769
The most informative terms for pos are: ['ham', 'iago', 'him', 'our', 'othello', 'what', 'his', 'lear', 'imogen', 'brutus', 'rom', 'nurse', 'romeo', 'caesar', 'thy', 'cassio', 'to', 'timon', 'posthumus', 'desdemona']
The most informative terms for neg are: ['you', 'prospero', 'duke', 'helena', 'antonio', 'me', 'for', 'your', 'sir', 'ariel', 'sebastian', 'herm

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [ ]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.730769
The most informative terms for pos are: ['the gods', 'art thou', 'the moor', 'lady macbeth', 'my lord', 'to the', 'give me', 'exeunt scene', 'ragged stones', 'ragged sails', 'ragged old', 'ragged know', 'ragged fearful', 'ragged entrails', 'ragg horns', 'rageth in', 'ragged man', 'raggedness defend', 'raging fire', 'rages upon']
The most informative terms for neg are: ['which is', 'me and', 'if you', 'she is', 'my love', 'to me', 'you have', 'the duke', 'in the', 'there is', 'the king', 'sir toby', 'of syracuse', 'it is', 'first lord', 'raggedness defend', '10 another', 'ragged sails', 'ragged old', 'ragged know']
----------L2 Norm-----------
The model's average accuracy is 0.769231
The most informative terms for pos are: ['my lord', 'to the', 'the gods', 'lady macbeth', 'the people', 'good night', 'with his', 'of rome', 'of our', 'the moor', 'art thou', 'first senator', 'to thee', 'give me', 'know not', 'the emperor